<a href="https://colab.research.google.com/github/aryanwalia2003/content_based_reccomnedation_system/blob/main/reccomendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata

 79% 7.00M/8.89M [00:00<00:00, 13.4MB/s]
100% 8.89M/8.89M [00:00<00:00, 10.6MB/s]


In [4]:
import zipfile

zip_ref = zipfile.ZipFile("/content/tmdb-movie-metadata.zip", "r")
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
credits=pd.read_csv("/content/tmdb_5000_credits.csv")

In [6]:
movies=pd.read_csv("/content/tmdb_5000_movies.csv")

In [7]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [8]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [9]:
credits.rename(columns={"movie_id":"id"},inplace=True)

In [10]:
credits.head(1)

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [11]:
print(credits.shape)
print(movies.shape)

(4803, 4)
(4803, 20)


In [12]:
movies.merge(credits,on="id").shape

(4803, 23)

In [13]:
df=movies.merge(credits,on="id")

In [14]:
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [15]:
df.head(1).tagline

0    Enter the World of Pandora.
Name: tagline, dtype: object

In [16]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')

In [17]:
#columns to keep - genres, id,keywords,overview,title_x,cast,crew

In [18]:
df=df[["id","title_x","genres","keywords","overview","cast","crew"]]

In [19]:
df.rename(columns={"title_x":"title"}, inplace=True)

<ipython-input-19-3e98ac8c4cf2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"title_x":"title"}, inplace=True)


In [20]:
df.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [21]:
df.isna().sum()

id          0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [22]:
df=df.dropna()

In [23]:
df.isna().sum()

id          0
title       0
genres      0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

In [24]:
df.duplicated().sum()

0

In [25]:
df.shape

(4800, 7)

In [26]:
type(df.genres)

pandas.core.series.Series

In [27]:
df.loc[df.title=='The Book of Mormon Movie, Volume 1: The Journey']

,id,title,genres,keywords,overview,cast,crew
4105,48382,"The Book of Mormon Movie, Volume 1: The Journey",[],[],The story of Lehi and his wife Sariah and thei...,"[{""cast_id"": 1, ""character"": ""Sam"", ""credit_id...",[]


In [28]:
df.head(1).genres.values

array(['[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'],
      dtype=object)

In [29]:
df.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [30]:
df.head(1).crew.values[0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [31]:
import json
json_string = '[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'
# Parse the JSON string into a list of dictionaries
keyword_list = json.loads(json_string)
names_list = []
for keyword in keyword_list:
    names_list.append(keyword['name'])
print(names_list)

['culture clash', 'future', 'space war', 'space colony', 'society', 'space travel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alien planet', 'cgi', 'marine', 'soldier', 'battle', 'love affair', 'anti war', 'power relations', 'mind and soul', '3d']


In [32]:
def extract_names(json_string):
    keyword_list = json.loads(json_string)
    return [entry['name'] for entry in keyword_list]

In [33]:
df.keywords=df.keywords.apply(extract_names)

In [34]:
df.genres=df.genres.apply(extract_names)

In [35]:
def extract_cast(obj):
  obj=json.loads(obj)
  actor_list=[]
  counter=0
  for i in obj:
    if counter!=3:
      actor_list.append(i["name"])
      counter=counter+1
    else:
      break
  return actor_list

In [36]:
df['cast'] = df['cast'].apply(extract_cast)

In [37]:
def extract_director(json_string):
    keyword_list = json.loads(json_string)
    for entry in keyword_list:
        if entry['job'] == 'Director':
            return entry['name']
    return None


In [38]:
df["director"]=df.crew.apply(extract_director)

In [39]:
df.head(1)

,id,title,genres,keywords,overview,cast,crew,director
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",James Cameron


In [40]:
df.drop(columns={"crew"},inplace=True)

In [41]:
df.genres=df.genres.apply(lambda x:[i.replace(" ","") for i in x])

In [42]:
df.isna().sum()

id           0
title        0
genres       0
keywords     0
overview     0
cast         0
director    30
dtype: int64

In [43]:
df.dropna(inplace=True)

In [44]:
df.cast=df.cast.apply(lambda x:[i.replace(" ","") for i in x])
df['director'] = df['director'].str.replace(" ", "")
df.keywords=df.keywords.apply(lambda x:[i.replace(" ","") for i in x])

In [45]:
df.head(1)

,id,title,genres,keywords,overview,cast,director
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","In the 22nd century, a paraplegic Marine is di...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",JamesCameron


In [46]:
df.isna().sum()

id          0
title       0
genres      0
keywords    0
overview    0
cast        0
director    0
dtype: int64

In [47]:
print(type(df.director))

<class 'pandas.core.series.Series'>


In [48]:
df.overview=df.overview.apply(lambda x:x.split())

In [49]:
df.head(1)

,id,title,genres,keywords,overview,cast,director
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",JamesCameron


In [50]:
df['director'] = df['director'].apply(lambda x: [x])

In [51]:
df["tags"]=df.overview+df.genres+df.keywords+df.cast+df.director

In [52]:
df.drop(columns=['genres', 'cast', 'director', 'keywords',"overview"], inplace=True)

In [53]:
df.sample().tags

2086    [Instead, of, flying, to, Florida, with, his, ...
Name: tags, dtype: object

In [54]:
df.head(1).tags[0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron']

In [55]:
df['tags'] = df['tags'].apply(lambda x: " ".join(map(str, x)))

In [56]:
df.head()

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [57]:
df.tags=df.tags.apply(lambda x:x.lower())

In [58]:
df.head(1).tags[0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [59]:
df.shape

(4770, 3)

In [60]:
import string
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [61]:
def pre_proccessing(text):
  text=nltk.word_tokenize(text) #break into words
  L=[]
  for i in text: ##removing special characters
    if i.isalnum():
      L.append(i)
    text=L[:]
    L.clear()
    for i in text: ##stemming of words
      L.append(ps.stem(i))
  return " ".join(L)

In [62]:
df.tags=df.tags.apply(pre_proccessing)

In [63]:
df.head(1).tags.values

array(['in the 22nd centuri a parapleg marin is dispatch to the moon pandora on a uniqu mission but becom torn between follow order and protect an alien civil action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'],
      dtype=object)

TEXT VECTORISATION

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
tfid_vector=tfidf.fit_transform(df.tags).toarray()
tfid_vector.shape

(4770, 28111)

In [107]:
tfid_vector[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [65]:
vector=cv.fit_transform(df.tags).toarray()

In [66]:
vector.shape

(4770, 5000)

In [67]:
vector[1]

array([0, 0, 0, ..., 0, 0, 0])

In [68]:
cv.get_feature_names_out()

array(['007', '10', '100', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [69]:
from sklearn.metrics.pairwise import cosine_similarity

In [70]:
similarity=cosine_similarity(vector)

In [77]:
type(enumerate(similarity[0]))

enumerate

In [76]:
type(list(enumerate(similarity[0])))

list

In [86]:
df.loc[df.index==1213]

,id,title,tags
1213,440,Aliens vs Predator: Requiem,a sequel to 2004 alien predat the icon creatur...


In [91]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(2403, 0.2597056905574371),
 (539, 0.24715576637149034),
 (507, 0.24602771043141894),
 (1213, 0.24198875709123052),
 (1201, 0.24140227479263376)]

In [101]:
def reccomend(movie):
  movie_index=df.loc[df.title==movie].index[0]
  distances=similarity[movie_index]
  movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movies_list:
    print(df.iloc[i[0]].title)
  return

In [105]:
reccomend("Batman Begins")

The Dark Knight
Batman
The Dark Knight Rises
10th & Wolf
City By The Sea


In [110]:
similarity_tfid=cosine_similarity(tfid_vector)

In [111]:
similarity_tfid[0]

array([1.        , 0.03017964, 0.02921706, ..., 0.04849721, 0.01551271,
       0.00801252])

In [112]:
def reccomend_tfidf(movie):
  movie_index=df.loc[df.title==movie].index[0]
  distances=similarity_tfid[movie_index]
  movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movies_list:
    print(df.iloc[i[0]].title)
  return

In [116]:
reccomend_tfidf("Batman Begins")

The Dark Knight
The Dark Knight Rises
Batman
Batman Returns
Batman Forever
